In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np

In [34]:
df_preprocessed = pd.read_csv('C:\\Users\\Janek\\Downloads\\df_train_preprocessed.csv')
df_preprocessed

,Unnamed: 0,File,Text,Category,language
0,0,bb995ead-f65c-43e8-9d7f-771546d21651.tiff,"['parliament', 'wsrdd', 'scrambeer', 'engeyi',...",advertisement,en
1,1,2a476c78-ce1c-4535-831c-626635b69fef.tiff,"['gilvvmather', 'insertion', 'order', 'i6ken7n...",advertisement,en
2,2,7bbd6936-4f3d-4eb4-bb75-983cbca08f48.tiff,"['newport', '902', 'cosmopolitan', 'april', '1...",advertisement,en
3,3,62d852b8-25eb-48e0-8cfe-0091c4f2453e.tiff,"['goforthegołd', 'introduce', 'true', 'gqld', ...",advertisement,en
4,4,8c5a5c2b-c58d-4f7e-985d-236bcc69011e.tiff,"['what', 's', 'tobacoo', 'induisty', 'dong', '...",advertisement,en
...,...,...,...,...,...
10824,10824,3827a69b-52cb-4600-a970-5163322ac4d7.jpg,"['umowa', '0', 'dzielo', 'kukulowo', '0625', '...",umowa_o_dzielo,pl
10825,10825,671cca7a-fe43-4743-9c23-4c4a4ef5b810.jpg,"['umowa', 'odiflo', 'zawrzeć', 'dzień', '20221...",umowa_o_dzielo,pl
10826,10826,a09e8f97-35c0-4327-9ab8-dfc2b2052722.jpg,"['uowaodiflo', 'zawrzeć', 'dzień', '2020032', ...",umowa_o_dzielo,pl
10827,10827,42ede5e5-8b9c-48d7-8996-ca81e8408b90.jpg,"['umowa', '0', 'dziłlo', 'zawrzeć', 'dzień', '...",umowa_o_dzielo,pl


In [45]:
import numpy as np
import pandas as pd
from autocorrect import Speller
from langdetect import detect
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re


class Preprocessor:
    def __init__(self, df: pd.DataFrame) -> None:
        self.df = df

    def preprocess_text(self, text_col, lemmatize=False):
        self.df[text_col] = self.df[text_col].apply(lambda x: str(x))
        self.convert_text_to_lowercase_and_remove_punctation(text_col)
        self.detect_language(text_col)
        # self.autocorrect_words(text_col)
        if lemmatize:
            self.remove_stopwords_column(text_col)
            self.lemmatize_column(text_col)

    def preprocess_image(self, image_col):
        # TODO
        pass

    def detect_language(self, text_col: str):
        def _detect_with_ignore(text: str):
            try:
                lang = detect(text)
            except:
                lang = 'en'
            return lang

        self.df['language'] = self.df[text_col].apply(lambda x: _detect_with_ignore(x))

    def convert_text_to_lowercase_and_remove_punctation(self, text_col: str):
        def remove_punctuation(text):
            punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~=+'''
            for ele in text:
                if ele in punc:
                    text = text.replace(ele, "")
            return text
        self.df[text_col] = self.df[text_col].str.lower()
        self.df[text_col] = self.df[text_col].apply(lambda x: remove_punctuation(x))

    def autocorrect_words(self, text_col: str):
        languages = set(self.df['language'])
        for lang in languages:
            try:
                spell = Speller(lang=lang, only_replacements=True)
            except:
                spell = Speller(lang='en', only_replacements=True)
            self.df[text_col] = np.where(self.df['language'] == lang, self.df[text_col].apply(lambda x: spell(x)),
                                         self.df[text_col])

    def remove_stopwords_column(self, text_col):
        self.df[text_col] = self.df.apply(
            lambda row: self.remove_stopwords(text=row[text_col], language=row['language']),
            axis=1
        )

    def remove_stopwords(self, text, language='en'):
        if language == 'pl':
            with open('../../data/stopwords/stop_words_polish.txt', 'r', encoding='utf-8') as file:
                content = file.readlines()
                content = [line.strip() for line in content]
                stop_words = set(content)
        else:
            stop_words = set(stopwords.words('english'))

        word_tokens = word_tokenize(text)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        return ' '.join(filtered_sentence)



    def lemmatize_column(self, text_col):
        self.df[text_col] = self.df.apply(
            lambda row: self.lemmatize_text(text=row[text_col], language=row['language']),
            axis=1
        )

    def lemmatize_text(self, text, language='en'):
        if language == "pl":
            nlp = spacy.load("pl_core_news_sm")
        else:
            nlp = spacy.load("en_core_web_sm")
        doc = nlp(text)
        return str([token.lemma_ for token in doc])


In [57]:
def preprocess_text(text):
    df = pd.DataFrame({'Text': [text]})
    preprocessor = Preprocessor(df)
    preprocessor.preprocess_text('Text', lemmatize=True)
    df_preprocessed = preprocessor.df
    return df_preprocessed

In [58]:
text='dsie a dsfajJSJF FJSF j!=d=sa'
df = pd.DataFrame({'Text': [text]})

In [59]:
preprocess_text(text)

,Text,language
0,"['dsie', 'dsfajjsjf', 'fjsf', 'jdsa']",da


In [48]:
aaa=Preprocessor(df)

In [49]:
aaa.preprocess_text('Text', lemmatize=True)

In [50]:
aaa.df

,Text,language
0,"['dsie', 'dsfajjsjf', 'fjsf', 'jdsa']",da


In [46]:
from sklearn.preprocessing import LabelEncoder
Encoder = LabelEncoder()
df_preprocessed['Category'] = Encoder.fit_transform(df_preprocessed['Category'])

In [6]:
df_preprocessed

,Unnamed: 0,File,Text,Category,language
0,0,bb995ead-f65c-43e8-9d7f-771546d21651.tiff,"['parliament', 'wsrdd', 'scrambeer', 'engeyi',...",0,en
1,1,2a476c78-ce1c-4535-831c-626635b69fef.tiff,"['gilvvmather', 'insertion', 'order', 'i6ken7n...",0,en
2,2,7bbd6936-4f3d-4eb4-bb75-983cbca08f48.tiff,"['newport', '902', 'cosmopolitan', 'april', '1...",0,en
3,3,62d852b8-25eb-48e0-8cfe-0091c4f2453e.tiff,"['goforthegołd', 'introduce', 'true', 'gqld', ...",0,en
4,4,8c5a5c2b-c58d-4f7e-985d-236bcc69011e.tiff,"['what', 's', 'tobacoo', 'induisty', 'dong', '...",0,en
...,...,...,...,...,...
10824,10824,3827a69b-52cb-4600-a970-5163322ac4d7.jpg,"['umowa', '0', 'dzielo', 'kukulowo', '0625', '...",19,pl
10825,10825,671cca7a-fe43-4743-9c23-4c4a4ef5b810.jpg,"['umowa', 'odiflo', 'zawrzeć', 'dzień', '20221...",19,pl
10826,10826,a09e8f97-35c0-4327-9ab8-dfc2b2052722.jpg,"['uowaodiflo', 'zawrzeć', 'dzień', '2020032', ...",19,pl
10827,10827,42ede5e5-8b9c-48d7-8996-ca81e8408b90.jpg,"['umowa', '0', 'dziłlo', 'zawrzeć', 'dzień', '...",19,pl


In [20]:
from sklearn.model_selection import train_test_split
df_train, df_val = train_test_split(df_preprocessed, test_size=0.2, random_state=42)

In [8]:
X_train = df_train['Text']
y_train = df_train['Category']
X_val = df_val['Text']
y_val = df_val['Category']
X_train_tokens= [str(word_tokenize(entry)) for entry in X_train]
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(X_train_tokens)
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Val_X_Tfidf = Tfidf_vect.transform(X_val)

## Naive Bayes

In [9]:
def training_pipeline_bayes(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):
    Naive = naive_bayes.MultinomialNB()
    Naive.fit(Train_X_Tfidf, y_train)
    predictions_NB = Naive.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_NB, y_val)
    f1 = f1_score(predictions_NB, y_val, average='macro')
    print("Naive Bayes Accuracy Score:", accuracy * 100)
    print("Naive Bayes F1 Score:", f1 * 100)
    return predictions_NB

In [10]:
training_pipeline_bayes(Train_X_Tfidf, Val_X_Tfidf, y_val)

TypeError: training_pipeline_bayes() missing 1 required positional argument: 'y_val'

## SVM

In [32]:
def training_pipeline_svm(df_train, df_val):
    X_train = df_train['Text']
    y_train = df_train['Category']
    X_val = df_val['Text']
    y_val = df_val['Category']
    X_train_tokens= [str(word_tokenize(entry)) for entry in X_train]
    Tfidf_vect = TfidfVectorizer(max_features=5000)
    Tfidf_vect.fit(X_train_tokens)
    Train_X_Tfidf = Tfidf_vect.transform(X_train)
    Val_X_Tfidf = Tfidf_vect.transform(X_val)
    SVM = svm.SVC(C=10, gamma='scale', kernel='rbf')
    SVM.fit(Train_X_Tfidf,y_train)
    predictions_SVM = SVM.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_SVM, y_val)
    f1 = f1_score(predictions_SVM, y_val, average='macro')
    precision = precision_score(predictions_SVM, y_val, average='macro')
    recall = recall_score(predictions_SVM, y_val, average='macro')
    confusion_mat = confusion_matrix(predictions_SVM, y_val)
    confusion_df = pd.DataFrame(confusion_mat, columns=np.unique(y_val), index=np.unique(y_val))
    print("SVM Accuracy Score:", accuracy * 100)
    print("SVM:", f1 * 100)
    print("SVM Precision:", precision * 100)
    print("SVM Recall:", recall * 100)
    print("Confusion Matrix:")
    print(confusion_df)
    return predictions_SVM

In [33]:
training_pipeline_svm(df_train, df_val)

SVM Accuracy Score: 73.08402585410896
SVM: 75.78342606923688
SVM Precision: 75.60363359300449
SVM Recall: 77.09002753157172
Confusion Matrix:
    0   1   2   3   4    5   6   7   8   9   ...  11  12  13   14  15  16  17  \
0   51   0   1   4   0    8   1   2   0   3  ...   0   7   2    0   0   7   0   
1    6  67   3   2   5    1  12   0   2   5  ...   0   5   4    0   0   6   2   
2    1   0  91   0   0    0   0   1   3   0  ...   0   3   0    0   0   0   0   
3   13   5   1  89   1    9   0   1   0   4  ...   0   8   0    0   1   3   2   
4    3   3   1   0  60    1   4   6   2   0  ...   0   2   3    0   0   8   2   
5   25   4   1   7   8  102  10   4   3  14  ...   0   9  13    1   3  11  11   
6    1  10   0   0   5    1  77   3   1   1  ...   0   2   2    1   0   2   1   
7    1   0   4   0   2    1   0  66   7   1  ...   0   4   4    0   0   4   0   
8    0   3   6   1   2    0   1  16  72   1  ...   0   3   3    1   0   4   1   
9    6   2   0   1   0    0   0   2   0  69  ...

array([ 6,  3,  5, ..., 16, 10,  5])

In [15]:
from sklearn.model_selection import GridSearchCV

def training_pipeline_svm(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):
    parameters = {'C': [1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']}
    SVM = svm.SVC()
    grid_search = GridSearchCV(SVM, parameters, cv=5)
    grid_search.fit(Train_X_Tfidf, y_train)
    
    best_SVM = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    best_SVM.fit(Train_X_Tfidf, y_train)
    predictions_SVM = best_SVM.predict(Val_X_Tfidf)
    
    accuracy = accuracy_score(predictions_SVM, y_val)
    f1 = f1_score(predictions_SVM, y_val, average='macro')
    precision = precision_score(predictions_SVM, y_val, average='macro')
    recall = recall_score(predictions_SVM, y_val, average='macro')
    confusion_mat = confusion_matrix(predictions_SVM, y_val)
    confusion_df = pd.DataFrame(confusion_mat, columns=np.unique(y_val), index=np.unique(y_val))
    
    print("Best Parameters:", best_params)
    print("SVM Accuracy Score:", accuracy * 100)
    print("SVM F1 Score:", f1 * 100)
    print("SVM Precision:", precision * 100)
    print("SVM Recall:", recall * 100)
    print("Confusion Matrix:")
    print(confusion_df)
    
    return predictions_SVM

In [16]:
training_pipeline_svm(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val)

Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
SVM Accuracy Score: 73.08402585410896
SVM F1 Score: 75.78342606923688
SVM Precision: 75.60363359300449
SVM Recall: 77.09002753157172
Confusion Matrix:
    0   1   2   3   4    5   6   7   8   9   ...  11  12  13   14  15  16  17  \
0   51   0   1   4   0    8   1   2   0   3  ...   0   7   2    0   0   7   0   
1    6  67   3   2   5    1  12   0   2   5  ...   0   5   4    0   0   6   2   
2    1   0  91   0   0    0   0   1   3   0  ...   0   3   0    0   0   0   0   
3   13   5   1  89   1    9   0   1   0   4  ...   0   8   0    0   1   3   2   
4    3   3   1   0  60    1   4   6   2   0  ...   0   2   3    0   0   8   2   
5   25   4   1   7   8  102  10   4   3  14  ...   0   9  13    1   3  11  11   
6    1  10   0   0   5    1  77   3   1   1  ...   0   2   2    1   0   2   1   
7    1   0   4   0   2    1   0  66   7   1  ...   0   4   4    0   0   4   0   
8    0   3   6   1   2    0   1  16  72   1  ...   0   3  

array([ 6,  3,  5, ..., 16, 10,  5])

bezbledna predykcja:
- umowa_sprzedazy_samochodu 
- umowa_o_dzielo 
- umowa_na_odleglosc_odstapienie 
- pozwolenie_uzytkowanie_obiektu_budowlanego

## XGBoost

In [ ]:
import xgboost as xgb

def training_pipeline_xgb(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):  
    xgb_model = xgb.XGBClassifier()
    xgb_model.fit(Train_X_Tfidf, y_train)
    
    predictions_xgb = xgb_model.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_xgb, y_val)
    f1 = f1_score(predictions_xgb, y_val, average='macro')
    
    print("XGBoost Accuracy Score:", accuracy * 100)
    print("XGBoost F1 Score:", f1 * 100)
    
    return predictions_xgb

In [ ]:
training_pipeline_xgb(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):

## LinearSVC

In [ ]:
from sklearn.svm import LinearSVC

def training_pipeline_linear(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):  
    linear_model = LinearSVC()
    linear_model.fit(Train_X_Tfidf, y_train)
    
    predictions_linear = linear_model.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_linear, y_val)
    f1 = f1_score(predictions_linear, y_val, average='macro')
    
    print("Linear Classifier Accuracy Score:", accuracy * 100)
    print("Linear Classifier F1 Score:", f1 * 100)
    
    return predictions_linear

In [91]:
training_pipeline_linear(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):

Linear Classifier Accuracy Score: 71.37580794090489
Linear Classifier F1 Score: 73.74887330222634


array([ 6,  3,  3, ..., 14, 10,  6])

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def training_pipeline_rf(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):
    rf_model = RandomForestClassifier()
    rf_model.fit(Train_X_Tfidf, y_train)
    
    predictions_rf = rf_model.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_rf, y_val)
    f1 = f1_score(predictions_rf, y_val, average='macro')
    
    print("Random Forest Accuracy Score:", accuracy * 100)
    print("Random Forest F1 Score:", f1 * 100)
    
    return predictions_rf

In [93]:
training_pipeline_rf(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):

Random Forest Accuracy Score: 69.62142197599262
Random Forest F1 Score: 72.21017976314799


array([ 4,  3,  3, ..., 14, 10,  5])

## Logistic Regression

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

def training_pipeline_lr(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):
    LR = LogisticRegression()
    LR.fit(Train_X_Tfidf, y_train)
    predictions_LR = LR.predict(Val_X_Tfidf)
    accuracy = accuracy_score(predictions_LR, y_val)
    f1 = f1_score(predictions_LR, y_val, average='macro')
    precision = precision_score(predictions_LR, y_val, average='macro')
    recall = recall_score(predictions_LR, y_val, average='macro')
    confusion_mat = confusion_matrix(predictions_LR, y_val)
    
    # Convert confusion matrix to DataFrame for better visualization
    confusion_df = pd.DataFrame(confusion_mat, columns=np.unique(y_val), index=np.unique(y_val))
    
    print("Logistic Regression Accuracy Score:", accuracy * 100)
    print("Logistic Regression F1 Score:", f1 * 100)
    print("Logistic Regression Precision:", precision * 100)
    print("Logistic Regression Recall:", recall * 100)
    
    return predictions_LR

In [2]:
training_pipeline_lr(Train_X_Tfidf, Val_X_Tfidf, y_train, y_val):

SyntaxError: invalid syntax (Temp/ipykernel_24848/2363693903.py, line 1)

# Top 2 logistic regression i SVM